In [ ]:
!pip install scikit-learn pandas numpy pyarrow

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from random import uniform, randint, sample
from imblearn.over_sampling import SMOTE
from abc import ABC, abstractmethod
from pandas import read_parquet
from copy import deepcopy
from math import ceil
from time import time

In [ ]:
class Individuo:
    def __init__(self, cromossomo):
        self.cromossomo = cromossomo
        self.media = 0

        self.acuracia = None
        self.precisao = None
        self.recall = None
        self.loss = None
        self.tempo_treinamento = None
        self.curva_loss = None
        self.curva_acuracia = None

    def __str__(self):
        return (
            f'Genes: {self.cromossomo} | '
            f'Acurácia: {self.acuracia}% | '
            f'Precisão: {self.precisao}% | '
            f'Recall: {self.recall}% | '
            f'Loss: {self.loss} | '
            f'Tempo: {self.tempo_treinamento:.2f}s | '
            f'Média: {self.media}%'
        )


In [ ]:
class Algoritmo(ABC):
    def __init__(self):
        self.min = self.definirMinimo()
        self.max = self.definirMaximo()
        self.qtd_genes = len(self.max)
        self.indices_float = self.definirIndices()

    @abstractmethod
    def __str__(self):
        pass

    @abstractmethod
    def definirMinimo(self):
        pass

    @abstractmethod
    def definirMaximo(self):
        pass

    @abstractmethod
    def gerarModelo(self, pos):
        pass

    def definirIndices(self):
        indices = []

        for i in range(self.qtd_genes):
            if not isinstance(self.min[i], int):
                indices.append(i)

        return indices

    def gerarCromossomo(self):
        cromossomo = []

        for i in range(self.qtd_genes):
            if i in self.indices_float:
                cromossomo.append(round(uniform(self.min[i], self.max[i]), 4))
            else:
                cromossomo.append(randint(self.min[i], self.max[i]))

        return cromossomo

    def gerarGene(self, i):
        if i in self.indices_float:
            return round(uniform(self.min[i], self.max[i]), 4)

        return randint(self.min[i], self.max[i])


In [ ]:
class MLP(Algoritmo):
    def __init__(self):
        self.qtd_camadas_ocultas = self.definirNumeroCamadasOcultas()
        super().__init__()

    @abstractmethod
    def definirNumeroCamadasOcultas(self):
        pass

    def definirMinimo(self):
        return [0, 0.0001, 0.0001, 128] + [16] * self.qtd_camadas_ocultas

    def definirMaximo(self):
        return [3, 1, 1, 512] + [256] * self.qtd_camadas_ocultas

    def gerarModelo(self, genes):
        activation = ["identity", "logistic", "tanh", "relu"]

        return MLPClassifier(
            activation=activation[genes[0]],
            learning_rate_init=genes[1],
            alpha=genes[2],
            batch_size=genes[3],
            hidden_layer_sizes=tuple(genes[4:]),
            early_stopping=True,
            n_iter_no_change=20,
            max_iter=300,
            solver="adam",
        )


In [ ]:
class MLP1(MLP):
    def __str__(self):
        return "MLP - 1 CAMADA OCULTA"

    def definirNumeroCamadasOcultas(self):
        return 1

In [ ]:
class MLP2(MLP):
    def __str__(self):
        return "MLP - 2 CAMADAS OCULTAS"

    def definirNumeroCamadasOcultas(self):
        return 2

In [ ]:
class MLP3(MLP):
    def __str__(self):
        return "MLP - 3 CAMADAS OCULTAS"

    def definirNumeroCamadasOcultas(self):
        return 3

In [ ]:
def definirAlgoritmo(tipo_algoritmo):
    if tipo_algoritmo == "MLPClassifier1":
        return MLP1()
    if tipo_algoritmo == "MLPClassifier2":
        return MLP2()
    if tipo_algoritmo == "MLPClassifier3":
        return MLP3()

    return None


def selecionarPorTorneio(populacao):
    pai1, pai2, mae1, mae2 = sample(populacao, 4)

    if pai1.media > pai2.media:
        pai = pai1
    else:
        pai = pai2

    if mae1.media > mae2.media:
        mae = mae1
    else:
        mae = mae2

    return deepcopy(pai), deepcopy(mae)

def calcularMedia(metricas):
  soma = 0

  for metrica in metricas:
      if metrica == 0.0:
          return 0.0
      else:
          soma += 1 / metrica

  return round(len(metricas) / soma, 1)


In [ ]:
class GA:
    def __init__(self, num_individuos, num_populacoes, tipo_algoritmo):
        self.num_individuos = num_individuos
        self.num_populacoes = num_populacoes
        self.algoritmo = definirAlgoritmo(tipo_algoritmo)
        self.qtd_genes = self.algoritmo.qtd_genes
        self.chance_de_mutar = 1 / self.qtd_genes
        self.melhor_individuo = Individuo(None)

        self.x_treinamento = None
        self.y_treinamento = None
        self.x_validacao = None
        self.y_validacao = None
        self.x_teste = None
        self.y_teste = None

    def definirXY(self, endereco_parquet):
        df = read_parquet(endereco_parquet)
        scaler = MinMaxScaler()
        x = df.iloc[:, :-1]
        y = df.iloc[:, -1]

        x_train, x_rest, y_train, y_rest = train_test_split(
            x, y, test_size=0.3, random_state=42, stratify=y)

        x_val, x_test, y_val, y_test = train_test_split(
            x_rest, y_rest, test_size=0.5, random_state=42, stratify=y_rest)

        self.x_treinamento = scaler.fit_transform(x_train)
        self.x_validacao = scaler.transform(x_val)
        self.x_teste = scaler.transform(x_test)

        self.y_treinamento = y_train.values
        self.y_validacao = y_val.values
        self.y_teste = y_test.values

    def gerarPopulacao(self):
        populacao = []

        for _ in range(self.num_individuos):
            populacao.append(Individuo(self.algoritmo.gerarCromossomo()))

        return populacao

    def retornarMedia(self, ind):
        modelo = self.algoritmo.gerarModelo(ind.cromossomo)

        inicio = time()
        modelo.fit(self.x_treinamento, self.y_treinamento)
        fim = time()

        ind.tempo_treinamento = fim - inicio

        previsoes = modelo.predict(self.x_validacao)
        probabilidades = modelo.predict_proba(self.x_validacao)

        ind.acuracia = round(
            accuracy_score(self.y_validacao, previsoes) * 100, 1
        )

        ind.precisao = round(
            precision_score(
                self.y_validacao, previsoes,
                zero_division=0, average='macro'
            ) * 100, 1
        )

        ind.recall = round(
            recall_score(
                self.y_validacao, previsoes,
                zero_division=0, average='macro'
            ) * 100, 1
        )

        ind.loss = round(
            log_loss(self.y_validacao, probabilidades), 4
        )

        if hasattr(modelo, "loss_curve_"):
            ind.curva_loss = modelo.loss_curve_
        else:
            ind.curva_loss = None

        if ind.curva_loss is not None:
            ind.curva_acuracia = [
                100 * (1 - loss) for loss in ind.curva_loss
            ]
        else:
            ind.curva_acuracia = None

        return calcularMedia([ind.acuracia, ind.precisao,ind.recall])

    def avaliarMelhorNoTeste(self, file):
        modelo = self.algoritmo.gerarModelo(self.melhor_individuo.cromossomo)
        modelo.fit(self.x_treinamento, self.y_treinamento)
        previsoes = modelo.predict(self.x_teste)

        acuracia = round(accuracy_score(self.y_teste, previsoes) * 100, 1)
        precisao = round(precision_score(self.y_teste, previsoes, zero_division=0, average='macro') * 100, 1)
        recall = round(recall_score(self.y_teste, previsoes, zero_division=0, average='macro') * 100, 1)
        media = calcularMedia([acuracia, precisao, recall])

        file.write("\nPerformance no Teste -> Acuracia: {}%, Precisao: {}%, Recall: {}%, Media: {}%".
                   format(acuracia, precisao, recall, media))

    def fazerCrossover(self, pai, mae):
        indices = sample(range(self.qtd_genes), randint(1, ceil(self.qtd_genes / 2)))

        for i in indices:
            pai.cromossomo[i], mae.cromossomo[i] = mae.cromossomo[i], pai.cromossomo[i]

        return pai, mae

    def mutar(self, filho, filha):
        for i in range(self.qtd_genes):
            if uniform(0, 1) <= self.chance_de_mutar:
                filho.cromossomo[i] = self.algoritmo.gerarGene(i)
            if uniform(0, 1) <= self.chance_de_mutar:
                filha.cromossomo[i] = self.algoritmo.gerarGene(i)

        filho.media = self.retornarMedia(filho)
        filha.media = self.retornarMedia(filha)

        return [filho, filha]

    def comecarPrimeiroLoop(self, populacao, file):
        file.write("1* Populacao:\n")

        for individuo in populacao:
            individuo.media = self.retornarMedia(individuo)

            file.write(str(individuo) + '\n')

            if individuo.media > self.melhor_individuo.media:
                self.melhor_individuo = deepcopy(individuo)

        file.write('\nMelhores ' + str(self.melhor_individuo) + "\n\n")

    def comecarDemaisLoops(self, populacao, file):
        for num_populacao in range(1, self.num_populacoes):
            file.write("{}* Populacao:\n".format(num_populacao + 1))
            nova_populacao = []

            for _ in range(self.num_individuos // 2):
                pai, mae = selecionarPorTorneio(populacao)
                filho, filha = self.fazerCrossover(pai, mae)
                irmaos = self.mutar(filho, filha)

                for irmao in irmaos:
                    file.write(str(irmao) + '\n')
                    nova_populacao.append(irmao)

                    if irmao.media > self.melhor_individuo.media:
                        self.melhor_individuo = deepcopy(irmao)

            file.write('\nMelhores ' + str(self.melhor_individuo) + "\n\n")
            populacao = nova_populacao

    def executar(self):
        populacao = self.gerarPopulacao()

        with open("resultados/" + str(self.algoritmo) + ".txt", "w") as file:
            self.comecarPrimeiroLoop(populacao, file)
            self.comecarDemaisLoops(populacao, file)
            self.avaliarMelhorNoTeste(file)


In [ ]:
lista = [ "MLPClassifier1", "MLPClassifier2", "MLPClassifier3" ]

for item in lista:
    print(item)
    ga = GA(6, 5, item)
    ga.definirXY('/content/fashion_mnist.parquet')
    ga.executar()
